In [9]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style

## This sets the plot style
## to have a grid on a white background
set_style("whitegrid")

import pprint

## import model objects
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score
#from tree_helpers import SkNode, traversable_nodes

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error


In [10]:
aggregate=pd.read_csv('./aggregate.csv',engine='python')
aggregate.set_index('GEO_ID',inplace=True)
agg_train, agg_test = train_test_split(aggregate,test_size=0.2,shuffle=True,random_state=121)

In [11]:
def powerset(l):
    pl=[[]]
    for x in l:
        pl=pl+[y+[x] for y in pl]
    return pl[1:]

In [12]:
features=['white_pc','log_pop','urban', 'unemp_bac', 'log_urban_pop', 'pov_pc', 'unemp_rate', 'bac_deg', 'hs_deg', 'pop_18_30_pc', 'pop_60_up_pc']
results=['democrat','republican']
agg_train['unemp_bac']=agg_train['unemp_rate']*agg_train['bac_deg']
agg_train['log_pop']=np.log10(agg_train['total_pop'].apply(lambda x: max(x,0.1)))
agg_train['log_urban_pop']=np.log10((agg_train['urban']*agg_train['total_pop']).apply(lambda x: max(x,0.1)))
for x in agg_train.index:
    agg_train.loc[x,'winner'] = 'D' if agg_train.loc[x,'democrat'] > agg_train.loc[x,'republican'] else 'R'

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [27]:
kfold=KFold(n_splits=5,shuffle=True,random_state=121)
#allfeatures=powerset(features)
#mse=np.zeros(len(allfeatures))
i=0
mse=np.zeros(5)
for train_i, test_i in kfold.split(agg_train):
    train_var = agg_train.iloc[train_i]
    test_var = agg_train.iloc[test_i]
#    baseline=np.ones(len(test_var))*(train_var['democrat'].values.mean())
#    mse[-1] += mean_squared_error(baseline, test_var['democrat'])
#    for i,featureset in enumerate(allfeatures):       
#        lr=LinearRegression()
    rf = RandomForestRegressor(
            n_estimators = 500, # number of trees in ensemble
            #max_depth = 10, # max_depth of each tree
            min_samples_leaf = 5, 
            max_features = 2, # default is round(sqrt(num_features)), which in this case is 1.
            bootstrap= True, # sampling with replacement
            max_samples = 500, # number of training samples selected with replacement to build tree
            random_state = 121 # for consistency
        )
#        if(len(featureset))==1:
#            rf.fit(train_var[featureset].values.reshape(-1,1),train_var['democrat'])
#            mse[i] += mean_squared_error(rf.predict(test_var[featureset].values.reshape(-1,1)),test_var['democrat'])
#        else:
#            rf.fit(train_var[featureset].values,train_var['democrat'])
#            mse[i] += mean_squared_error(rf.predict(test_var[featureset].values),test_var['democrat'])
#        print(i)
    rf.fit(train_var[features].values,train_var['democrat'])
    #mse[i]+=mean_squared_error(rf.predict(test_var[features].values.reshape(-1,1),test_var['democrat']))
    mse[i] += mean_squared_error(rf.predict(test_var[features].values),test_var['democrat'])
    print(i)
    print(mse[i])
    i=i+1

0
85.22407918892796
1
82.96975418882809
2
90.2056232388459
3
76.4784683690093
4
84.581295024769


In [29]:
m=0
for i in range(0,len(mse)):
    m+=mse[i]
    print(i)
m=m/5
print(m)

0
1
2
3
4
83.89184400207606
